# 8.에러와 예외

두 가지 구별되는 에러들이 있습니다; 문법 에러 와 예외.

# 8.1 문법 에러

문법 에러(파싱 에러)

In [1]:
# print() 에서 감지되었는데, 그 앞에 콜론 (':') 이 빠져있기 때문
while True print('Hello world')

SyntaxError: ignored

에러는 화살표 앞에 오는 토큰이 원인입니다

# 8.2 예외

실행 중에 감지되는 에러들을 예외 라고 부르고 무조건 치명적이지는 않습니다.

In [ ]:
10 * (1/0)
4 + spam*3
'2' + 2

에러 메시지의 마지막 줄은 어떤 일이 일어났는지 알려줍니다. 

 ZeroDivisionError, NameError, TypeError

# 8.3 예외 처리하기

In [ ]:
while True:
  try:
    x = int(input("Please enter a number: "))
    break
  except ValueError:
    print("Oops! That was no valid number. Try again...")

In [ ]:
try:
  pass
except (RuntimeError, TypeError, NameError):
  pass

In [2]:
# 실행결과: B C D
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
C
D


In [3]:
# 순서가 바뀌면 BBB
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except B:
        print("B")
    except D:
        print("D")
    except C:
        print("C")


B
B
B


In [ ]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except BaseException as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

All exceptions inherit from BaseException, and so it can be used to serve as a wildcard.

 It can also be used to print an error message

In [ ]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

**[else절]:** try절이 예외를 일으키지않으면 무조건 실행괴어야하는 코드를 작성할 때 유용하다.


else 절의 사용이 try 절에 코드를 추가하는 것보다 좋은데, try … except 문에 의해 보호되고 있는 코드가 일으키지 않은 예외를 우연히 잡게 되는 것을 방지하기 때문입니다.

In [5]:
try:
  raise Exception('spam','eggs')
except Exception as inst:
  print(type(inst))
  print(inst.args)
  print(inst)

  x, y = inst.args
  print('x=', x)
  print('y=', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x= spam
y= eggs


In [ ]:
def this_fails():
  x=1/0

try:
  this_fails()
except ZeroDivisionError as err:
  print('Handling run-time error:', err)

# 8.4 예외 일으키기

raise 문은 프로그래머가 지정한 예외가 발생하도록 강제할 수 있게 합니다. 

In [6]:
raise NameError('HiThere')

NameError: ignored

In [7]:
raise ValueError

ValueError: ignored

In [8]:
try:
  raise NameError('HiTHere')
except NameError:
  print('An exception flew by!')
  raise

An exception flew by!


NameError: ignored

# 8.5 예외 연쇄

The raise statement allows an optional from which enables chaining exceptions. 

In [ ]:
# exc must be exception instance or None.
raise RuntimeError from exc

In [9]:
# 예외를 변환
def func():
  raise ConnectionError

try:
  func()
except ConnectionError as exc:
  raise RuntimeError('Failed to open database') from exc

RuntimeError: ignored

In [ ]:
# 이 경우에는 try문에서 발생한 OSError 메세지가 나오지않음
try:
  open('database.sqlite')
except OSError:
  raise RuntimeError from None

# 8.6 사용자 정의 예외

새 예외 클래스를 만듦으로써 프로그램은 자신의 예외에 이름을 붙일 수 있습니다.

대부분의 예외는 표준 예외들의 이름들과 유사하게, 《Error》 로 끝나는 이름으로 정의됩니다.

# 8.7 뒷정리 동작 정의하기

모든 상황에 실행되어야만 하는 뒷정리 동작

In [10]:
try:
  raise KeyboardInterrupt
finally:
  print('Goodbye, world!')
  

Goodbye, world!


KeyboardInterrupt: ignored

finally 절이 있으면, try 문이 완료되기 전에 finally 절이 마지막 작업으로 실행됩니다. finally 절은 try 문이 예외를 생성하는지와 관계없이 실행됩니다. 

In [ ]:
def bool_return():
  try:
    return True
  finally:
    return False

bool_return() # 실행결과: False

In [11]:
def divide(x, y):
  try:
    result = x / y
  except ZeroDivisionError:
    print('division by zero!')
  else:
    print("result is", result)
  finally:
    print("executing finalyl clause")


In [12]:
divide(2, 1)

# result is 2.0
# executing finalyl clause

result is 2.0
executing finalyl clause


In [13]:
divide(2, 0)

# division by zero!
# executing finalyl clause

division by zero!
executing finalyl clause


In [14]:
divide("2", "1")

# executing finalyl clause

executing finalyl clause


TypeError: ignored

# 8.8 미리 정의된 뒷정리 동작들

In [ ]:
for line in open("myfile.txt"):
    print(line, end="")

이 코드의 문제점은 이 부분이 실행을 끝낸 뒤에도 예측할 수 없는 기간 동안 파일을 열린 채로 둔다는 것입니다. 간단한 스크립트에서는 문제가 되지 않지만, 큰 응용 프로그램에서는 문제가 될 수 있습니다. with 문은 파일과 같은 객체들이 즉시 올바르게 뒷정리 되도록 보장하는 방법을 제공합니다.

In [ ]:
with open("myfile.txt") as f:
    for line in f:
        print(line, end="")

문장이 실행된 후에, 줄을 처리하는 데 문제가 발생하더라도, 파일 f 는 항상 닫힙니다. 